In [1]:
!pwd

/afs/cern.ch/work/a/apoyet/bbcw2mad/examples


In [2]:
!which python

/cvmfs/sft.cern.ch/lcg/views/LCG_96python3/x86_64-centos7-gcc8-opt/bin/python


# BBCW2MAD SANITY CHECKS
---

In this notebook, we follow the new approach proposed in [1] and install the beam-beam wire compensators on the LHC, testing the effect on the tunes and orbit. 

In [3]:
# %% Import few packages
from cpymad.madx import Madx
#import madxp
from madxp import cpymadTool as mt
from collections import OrderedDict
import numpy as np
import pandas as pd
from matplotlib import pylab as plt
from IPython.display import display
import time
import os
import warnings
warnings.filterwarnings('always')
from cl2pd import importData

#bbcw2mad package
from bbcw2mad.WireObj import *

#phd-tools
from phd_toolbox import constants
from phd_toolbox import wires_tools


In [18]:
# %%
# ============================================================================================================ #
#                                              WORKFLOW FUNCTIONS                                              #
# ============================================================================================================ #

def power_wires(mad,wires_lst,table_out):
    '''
    Function to turn on the wires as done in the LHC Run 3.
    
    Args:
        mad: madx instance
        wires_lst: list, contains the wires under the form of WireObj instances
        table_out: str, name of the madx table to be added to "twiss_" and "summary_" 
    Return: 
        my_output_dict: dict, contains the output info to be added to the working flow DF.
    '''
    start_time = time.time()
    
    # Turn off BB
    mad.input(f'''
    ON_BB_CHARGE=0;
    ''')

    for wire in wires_lst:
        wire.switch_wire(on=True)
        wire.print_lense()

    mad.input(f'''
    use, sequence=lhcb1;
    twiss, table=twiss_{table_out};
    exec, clonesumm('summary_{table_out}');
    !ON_BB_CHARGE=1;
    ''')
    
    my_output_dict = get_status(mad)
    elapsed_time = time.time() - start_time

    my_output_dict['elapsed_time'] = elapsed_time
    return my_output_dict

def get_status(mad):
    start_time = time.time()
    variables=mt.variables_dict(mad)
    my_output_dict= {'constant_df': variables['constant_df'],
            'independent_variable_df': variables['independent_variable_df'],
            'dependent_variable_df': variables['dependent_variable_df'],
            'sequences_df': mt.sequences_df(mad),
            'beams_df': mt.beams_df(mad),
            'tables_list': list(mad.table)}
    elapsed_time = time.time() - start_time
    my_output_dict['elapsed_time'] = elapsed_time
    return my_output_dict

## Checks on B1 
---

In [19]:
# %% Definition of the parameters that are not knobs of the beam sequence (no strings please!)
parameter_dict={
    # =============================================================================
    # Beam parameters
    # =============================================================================
    ## LHC beam 1 (clockwise), LHC beam 2 (clockwise), LHC beam 2 (counterclockwise) 
    'par_mylhcbeam': 1, 
    ## beam normalized emittance [m rad]
    'par_beam_norm_emit': 2.5e-6,
    ## [m]
    'par_beam_sigt': 0.075,
    ## [-]           
    'par_beam_sige': 1.1e-4,
    ## [-]                    
    'par_beam_npart': 1.1e11, 
    ## [GeV]            
    'par_beam_energy_tot': 7000,
    ## [A]          
    'par_oct_current': 350,
    ## [-]            
    'par_chromaticity': 15,
    ## [MV]          
    'par_vrf_total': 16.,
    ## Tunes with fractional part          
    'par_qx0': 62.31, 'par_qy0': 60.32,
    # =============================================================================
    # Beam-Beam configuration 
    # =============================================================================
    ## install the BB elements [0,1]
    'par_on_bb_switch': 1,
    ## if 1 lumi leveling in ip8 is applied and q/q' match is done with bb off [0,1]
    'par_on_collision': 1, 
    ## bunch separation [ns]               
    'par_b_t_dist': 25.,   
    ## default value for the number of additionnal parasitic encounters inside D1              
    'par_n_inside_D1': 5,                 
    ## number of slices for head-on in IR1 [between 0 and 201]
    'par_nho_IR1': 11, 'par_nho_IR2': 11, 'par_nho_IR5': 11, 'par_nho_IR8': 11, 
    ## flag to install the Crab Cavities [0, 1]
    'par_install_crabcavities': 0,
    # =============================================================================
    # Beam-Beam Wire Compensators 
    # =============================================================================
    ## IR1
    'par_s_w_ir1': 19848.217400,              # s-position [m]
    'par_x_w_ir1': 0,                         # Horizontal transverse position of the wire, wrt to the beam [m]
    'par_y_w_ir1': 0.0091,                    # Vertical transverse position of the wire, wrt to the beam [m]
    'par_I_w_ir1': 350,                       # Wire Current [A]
    ## IR5
    'par_s_w_ir5': 6516.623433,   # s-position [m]
    'par_x_w_ir5': 0.01223,   # Horizontal transverse position of the wire, wrt to the beam [m]
    'par_y_w_ir5': 0,   # Vertical transverse position of the wire, wrt to the beam [m]
    'par_I_w_ir5': 350,   # Wire Current [A]
    # =============================================================================
    # Leveling in IP8   
    # =============================================================================
    # leveled luminosity in IP8 (considered if par_on_collision=1) [Hz/cm2]
    'par_lumi_ip8': 2e32,                 
    # These variables define the number of Head-On collisions in the 4 IPs
    'par_nco_IP1': 2544, 'par_nco_IP2': 2215, 'par_nco_IP5': 2544, 'par_nco_IP8': 2332,
    # =============================================================================
    # Errors and corrections 
    # =============================================================================
    # Select seed for errors
    'par_myseed': 0,
    # Set this flag to correct the errors of D2 in the NLC 
    # (warning: for now only correcting b3 of D2, still in development)
    'par_correct_for_D2': 0,
    # Set this flag to correct the errors of MCBXF in the NLC 
    # (warning: this might be less reproducable in reality, use with care)
    'par_correct_for_MCBX': 0,
    'par_off_all_errors': 0,
    'par_on_errors_LHC': 0,
    'par_on_errors_MBH': 0,
    'par_on_errors_Q5': 0,
    'par_on_errors_Q4': 0,
    'par_on_errors_D2': 0,
    'par_on_errors_D1': 0,
    'par_on_errors_IT': 0,
    'par_on_errors_MCBRD': 0,
    'par_on_errors_MCBXF': 0,
    # =============================================================================
    # Additional parameters
    # =============================================================================
    # parameter for having verbose output [0,1]
    'par_verbose': 1,
    # definition of the slicefactor used in the makethin
    'par_slicefactor': 4,
    # number of optics to use
    'par_optics_number':27,
    # Specify machine version
    'ver_lhc_run' : 3, 'ver_hllhc_optics' : 0,
}


In [20]:
mad = Madx()


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2020.05.04 16:50:02      +
  ++++++++++++++++++++++++++++++++++++++++++++


In [21]:
mad.input('option, -echo,-warn,-info;')
mad.input('call, file=seq_b1.seq;')
mad.call('/afs/cern.ch/user/s/sterbini/public/tracking_tools/tools/macro.madx') 
mad.call('tools/optics_indep_macros.madx')

In [22]:
# Definition of the wire (ir1-like wire)
sequence_w = f'lhcb{parameter_dict["par_mylhcbeam"]}'
mad_inst_w = mad

s_w_ir1 = parameter_dict['par_s_w_ir1']
x_w_ir1 = parameter_dict['par_x_w_ir1']
y_w_ir1 = parameter_dict['par_y_w_ir1']
I_w_ir1 = parameter_dict['par_I_w_ir1']

my_bbcw_test = WireObj('bbcw_int_l1b1',I_w_ir1,s_w_ir1,x_w_ir1,y_w_ir1,parameter_dict['par_beam_npart'],mad_inst=mad_inst_w,sequence=sequence_w)

my_bbcw_test.describe()
wires_lst = [my_bbcw_test]
my_workflow_dict = OrderedDict()

{'current': 350,
 'mad_inst': <cpymad.madx.Madx object at 0x7f1da506e7f0>,
 'name': 'bbcw_int_l1b1',
 'nb': 110000000000.0,
 's_pos': 19848.2174,
 'sequence': 'lhcb1',
 'sigx': 0.0003333333333333333,
 'sigy': 0.0003333333333333333,
 'switch': 0,
 'x_co': 0,
 'x_pos': 0,
 'xma': 0,
 'y_co': 0,
 'y_pos': 0.0091,
 'yma': 0.0091}


In [23]:
mad.input('''
use, sequence=lhcb1;
twiss, table=twiss_after_machine_tuning;
exec, clonesumm('summary_after_machine_tuning');
''')

enter Twiss module
  
iteration:   1 error:   5.122208E-07 deltap:   0.000000E+00
orbit:   1.420940E-08  5.810138E-10 -3.224612E-08 -4.099646E-10 -6.521956E-08 -5.518409E-08

++++++ table: summ

            length             orbit5               alfa            gammatr 
        26658.8832    6.521956412e-08    0.0003481891584        53.59106378 

                q1                dq1            betxmax              dxmax 
        62.3099992        15.00109242        8093.846534        3.039960097 

             dxrms             xcomax             xcorms                 q2 
        1.41726492     0.007742819485     0.001013652373        60.31999917 

               dq2            betymax              dymax              dyrms 
       15.00790243        8097.308728        3.583153378        0.657213627 

            ycomax             ycorms             deltap            synch_1 
     0.00972582359     0.001801863231                  0                  0 

           synch_2            

True

In [24]:
twiss_before = mad.table.twiss_after_machine_tuning.dframe()
x_bef = twiss_before[twiss_before['name']=='bbcw_int_l1b1:1']['x'].values[0]
y_bef = twiss_before[twiss_before['name']=='bbcw_int_l1b1:1']['y'].values[0]
qx_bef = mad.table.summary_after_machine_tuning['q1'][0]
qy_bef = mad.table.summary_after_machine_tuning['q2'][0]

betx_w = twiss_before[twiss_before['name']=='bbcw_int_l1b1:1']['betx'].values[0]
bety_w = twiss_before[twiss_before['name']=='bbcw_int_l1b1:1']['bety'].values[0]

# Update wire position
for wire in wires_lst:
    wire.get_closed_orbit(table_input='twiss_after_machine_tuning')
mad.input('option, bborbit=true;')
my_workflow_dict['turn_on_wires'] = power_wires(mad,wires_lst,table_out='b1_wires')

twiss_after = mad.table.twiss_b1_wires.dframe()

x_aft = twiss_after[twiss_after['name']=='bbcw_int_l1b1:1']['x'].values[0]
y_aft = twiss_after[twiss_after['name']=='bbcw_int_l1b1:1']['y'].values[0]
qx_aft = mad.table.summary_b1_wires['q1'][0]
qy_aft = mad.table.summary_b1_wires['q2'][0]

dx = x_aft - x_bef
dy = y_aft - y_bef
dqx = qx_aft - qx_bef
dqy = qy_aft - qy_bef

enter Twiss module
  
iteration:   1 error:   7.462047E-05 deltap:   0.000000E+00
orbit:   5.292879E-08 -7.853671E-10 -8.389496E-05 -1.236550E-06 -3.184079E-08 -2.769724E-08
  
iteration:   2 error:   2.122107E-08 deltap:   0.000000E+00
orbit:   6.997913E-08 -1.077963E-09 -8.390748E-05 -1.236741E-06 -3.272704E-08 -2.761750E-08

++++++ table: summ

            length             orbit5               alfa            gammatr 
        26658.8832    3.272703819e-08    0.0003481907875        53.59093841 

                q1                dq1            betxmax              dxmax 
       62.31764043        13.55143782        8462.774908        3.044527018 

             dxrms             xcomax             xcorms                 q2 
       1.416951788     0.007742705163     0.001013670776        60.31670312 

               dq2            betymax              dymax              dyrms 
       15.21118845        8238.403905        3.625249862       0.6663543524 

            ycomax            

In [25]:
from cl2pd import particle

# theory 

dx_th = wires_tools.orbit_shift(I_L=my_bbcw_test.current, r_w=my_bbcw_test.y_pos, betx=betx_w, bety=bety_w, qx=qx_bef, qy=qy_bef, phi_w=np.pi/2, Brho=particle.setTotalEnergy_GeV(7000)['magneticRigidity_Tm'])[0]
dy_th = wires_tools.orbit_shift(I_L=my_bbcw_test.current, r_w=my_bbcw_test.y_pos, betx=betx_w, bety=bety_w, qx=qx_bef, qy=qy_bef, phi_w=np.pi/2, Brho=particle.setTotalEnergy_GeV(7000)['magneticRigidity_Tm'])[1]

dqx_th = wires_tools.tune_shift(I_L=my_bbcw_test.current, r_w=my_bbcw_test.y_pos, betx=betx_w, bety=bety_w, phi_w=np.pi/2, Brho=particle.setTotalEnergy_GeV(7000)['magneticRigidity_Tm'])[0]
dqy_th = wires_tools.tune_shift(I_L=my_bbcw_test.current, r_w=my_bbcw_test.y_pos, betx=betx_w, bety=bety_w, phi_w=np.pi/2, Brho=particle.setTotalEnergy_GeV(7000)['magneticRigidity_Tm'])[1]


In [26]:
# results

print('##### Orbit shift #####')
print('+++ From MAD +++')
print(f'dx = {dx*1e3} mm')
print(f'dy = {dy*1e3} mm')
print('+++ From formula +++')
print(f'dx = {dx_th*1e3} mm')
print(f'dy = {dy_th*1e3} mm')
print('+++ Error +++')
print(f'err_dx = {100*(dx-dx_th)/np.abs(dx_th)} %')
print(f'err_dy = {100*(dy-dy_th)/np.abs(dy_th)} %\n')

print('##### Tune shift #####')
print('+++ From MAD +++')
print(f'dqx = {dqx}')
print(f'dqy = {dqy}')
print('+++ From formula +++')
print(f'dqx = {dqx_th}')
print(f'dqy = {dqy_th}')
print('+++ Error +++')
print(f'err_dqx = {100*(dqx-dqx_th)/np.abs(dqx_th)} %')
print(f'err_dqy = {100*(dqy-dqy_th)/np.abs(dqy_th)} %')

##### Orbit shift #####
+++ From MAD +++
dx = 0.00023799976165095338 mm
dy = 0.116554820115494 mm
+++ From formula +++
dx = 1.738999474844759e-17 mm
dy = 0.11508584106018811 mm
+++ Error +++
err_dx = 1368601687888274.2 %
err_dy = 1.276420315282436 %

##### Tune shift #####
+++ From MAD +++
dqx = 0.007641229462038268
dqy = -0.003296052691844409
+++ From formula +++
dqx = 0.007308732629398052
dqy = -0.0031716471379402626
+++ Error +++
err_dqx = 4.549309018403655 %
err_dqy = -3.9224273222568495 %
